In [1]:
%cd drive/MyDrive/MyProjects/SSD/ssd

/content/drive/MyDrive/MyProjects/SSD/ssd


In [2]:
import sys
sys.path.append('content/drive/MyDrive/MyProjects/SSD/ssd')


In [3]:
from Utils.SSDLoss import SSDLoss
from Utils.DataGenerator import DataGenerator
from Model.SSDInceptionResnet import ssd_inceptionResnet


Go back to content

In [24]:
%cd ../ 

/content


In [9]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar

--2021-08-28 03:58:28--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  13.2MB/s    in 36s     

2021-08-28 03:59:04 (12.2 MB/s) - ‘VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]



In [11]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [12]:
!ls

dataset  drive	sample_data  VOCtrainval_06-Nov-2007.tar


In [13]:
cp VOCtrainval_06-Nov-2007.tar ./dataset

In [14]:
%cd dataset

/content/dataset


In [6]:
!ls

dataset  sample_data		      VOCtrainval_06-Nov-2007.tar.1
drive	 VOCtrainval_06-Nov-2007.tar  VOCtrainval_06-Nov-2007.tar.2


In [15]:
!tar xf VOCtrainval_06-Nov-2007.tar 

In [16]:
%cd VOCdevkit/VOC2007

/content/dataset/VOCdevkit/VOC2007


In [22]:
!ls

dataset  sample_data		      VOCtrainval_06-Nov-2007.tar.1
drive	 VOCtrainval_06-Nov-2007.tar  VOCtrainval_06-Nov-2007.tar.2


In [18]:
import matplotlib.pyplot as plt
img=plt.imread("JPEGImages/000005.jpg")
import xml.etree.ElementTree as ET
tree = ET.parse('Annotations/000005.xml')
root=tree.getroot()
obj=root.findall("object")
for obj in root.iter('object'):
  print(obj.tag)

object
object
object
object
object


In [17]:
voc_image_dir="JPEGImages/"
voc_annotation_dir="Annotations/"
voc_train_image_set_filename="ImageSets/Main/train.txt"
voc_val_image_set_filename="ImageSets/Main/val.txt"
classes =  ['background',
           'aeroplane'
           ]

train_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)

val_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)

train_dataset.parse_xml(images_dirs=voc_image_dir,
                        image_set_filenames=voc_train_image_set_filename,
                        annotations_dirs=voc_annotation_dir,
                        classes=classes,
                        include_classes='all',
                        exclude_truncated=False,
                        exclude_difficult=False,
                        ret=False)


Loading images into memory: 100%|██████████| 2501/2501 [00:11<00:00, 217.81it/s]


In [18]:
img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 20 # Number of positive classes, e.g. 20 for Pascal VOC, 80 for MS COCO
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # The anchor box scaling factors used in the original SSD300 for the Pascal VOC datasets
scales_coco = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets
scales = scales_pascal
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True

In [19]:
model = ssd_inceptionResnet(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                swap_channels=swap_channels)


In [20]:
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

In [21]:
model.compile(optimizer='adam', 
      loss=ssd_loss.compute_loss,
              metrics=['accuracy'])
model.fit(train_dataset,epochs=1)

ValueError: ignored

In [2]:
!ls

BaseNet  Inference  layers  Model  Utils


In [27]:
!git init

Initialized empty Git repository in /content/drive/My Drive/MyProjects/SSD/ssd/.git/


In [29]:
!git reset

In [3]:
!git add *

fatal: Unable to create '/content/drive/MyDrive/MyProjects/SSD/ssd/.git/index.lock': File exists.

Another git process seems to be running in this repository, e.g.
an editor opened by 'git commit'. Please make sure all processes
are terminated then try again. If it still fails, a git process
may have crashed in this repository earlier:
remove the file manually to continue.


In [ ]:
git commit -m "First commit"